In [1]:
//input
val fileName1 = "dataset_abstracts/a9000006.txt"
val fileName2 = "dataset_abstracts/a9000031.txt"

val rows1 = sc.textFile(fileName1).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text1 = rows1.reduce(_ + _)

val rows2 = sc.textFile(fileName2).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text2 = rows2.reduce(_ + _)

fileName1 = dataset_abstracts/a9000006.txt
fileName2 = dataset_abstracts/a9000031.txt
rows1 = MapPartitionsRDD[2] at map at <console>:31
text1 = world-wide distribution, but the Atlantic and Pacificpopulations of the northern hemisphere appear to be discretepopulations, as is the population of the southern hemisphericoceans. Each of these oceanic populations may be furthersubdivided into smaller isolates, each with its own migratorypattern and somewhat distinct gene pool. This study willprovide information on the level of genetic isolation amongpopulations and the levels of gene flow and genealogicalrelationships among populations. This detailed geneticinformation will facilitate international policy decisionsregarding the conser...


world-wide distribution, but the Atlantic and Pacificpopulations of the northern hemisphere appear to be discretepopulations, as is the population of the southern hemisphericoceans. Each of these oceanic populations may be furthersubdivided into smaller isolates, each with its own migratorypattern and somewhat distinct gene pool. This study willprovide information on the level of genetic isolation amongpopulations and the levels of gene flow and genealogicalrelationships among populations. This detailed geneticinformation will facilitate international policy decisionsregarding the conser...

[Ljava.lang.String;@7bf68b88
[Ljava.lang.String;@23c3587d
[Ljava.lang.String;@6a2ae00b
[Ljava.lang.String;@61627042
[Ljava.lang.String;@48008a6e
[Ljava.lang.String;@18080d10
[Ljava.lang.String;@63f720cb
[Ljava.lang.String;@7714d999
[Ljava.lang.String;@2964ec17
[Ljava.lang.String;@2cf3999e
[Ljava.lang.String;@300bb5ea
[Ljava.lang.String;@657b50af
[Ljava.lang.String;@687f8e3d
[Ljava.lang.String;@3d43a19d
[Ljava.lang.String;@39f64aba
[Ljava.lang.String;@59c2e7f6
[Ljava.lang.String;@67e83f86
[Ljava.lang.String;@9f91b73
[Ljava.lang.String;@6bb3c5cb
[Ljava.lang.String;@69627338
[Ljava.lang.String;@622ca210
[Ljava.lang.String;@56a3a893
[Ljava.lang.String;@593dcf28
[Ljava.lang.String;@764ca493
[Ljava.lang.String;@178e6eae
[Ljava.lang.String;@50fea1d3
[Ljava.lang.String;@62e66c09
[Ljava.lang.String;@2a7fe347
[Ljava.lang.String;@655d6e67
[Ljava.lang.String;@61d331ab
[Ljava.lang.String;@52aea96b
[Ljava.lang.String;@7228693e
[Ljava.lang.String;@3631568e
[Ljava.lang.String;@577dbe91
[Ljava.lang.Str

In [2]:
//k-gram
val k = 5
val k_gram1 = text1.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD1 = sc.parallelize(k_gram1)
val filtered1 = kgramRDD1.distinct()

val k_gram2 = text2.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD2 = sc.parallelize(k_gram2)
val filtered2 = kgramRDD2.distinct()


k = 5
k_gram1 = List((-679493040,""), (2019085497,""), (-1450498693,""), (569023187,""), (997395134,""), (-2085731237,""), (-1231717628,""), (-377010459,""), (-1101467996,""), (-572863609,""), (-1452639189,""), (1059127100,""), (-387969419,""), (-585145512,""), (496721826,""), (342454299,""), (2015323395,""), (71880523,""), (2023557668,""), (1046643096,""), (-1132943064,""), (-903658973,""), (-1778189634,""), (-1637413437,""), (1062705570,""), (650570931,""), (-1597084207,""), (499302821,""), (1003038978,""), (-126474325,""), (-735860716,""), (-1767237806,""), (1925341296,""), (44303747,""), (-931668577,""), (-228801676,""), (359871299,""), (-9619924,""), (997351251,""), (1000521331,""), (279433434,""), (1597990745,""), (-31869785,""), (-2141757007,""), (-11583...


List((-679493040,""), (2019085497,""), (-1450498693,""), (569023187,""), (997395134,""), (-2085731237,""), (-1231717628,""), (-377010459,""), (-1101467996,""), (-572863609,""), (-1452639189,""), (1059127100,""), (-387969419,""), (-585145512,""), (496721826,""), (342454299,""), (2015323395,""), (71880523,""), (2023557668,""), (1046643096,""), (-1132943064,""), (-903658973,""), (-1778189634,""), (-1637413437,""), (1062705570,""), (650570931,""), (-1597084207,""), (499302821,""), (1003038978,""), (-126474325,""), (-735860716,""), (-1767237806,""), (1925341296,""), (44303747,""), (-931668577,""), (-228801676,""), (359871299,""), (-9619924,""), (997351251,""), (1000521331,""), (279433434,""), (1597990745,""), (-31869785,""), (-2141757007,""), (-11583...

In [3]:

val join = filtered1.leftOuterJoin(filtered2)
val join_n = join.count().toFloat
val union = filtered1.union(filtered2).distinct()
val union_n = union.count().toFloat

val sim = join_n / union_n
val jacc_dist = 1-sim

println(sim)
println(jacc_dist)


0.51371205
0.48628795


join = MapPartitionsRDD[16] at leftOuterJoin at <console>:28
join_n = 1049.0
union = MapPartitionsRDD[20] at distinct at <console>:30
union_n = 2042.0
sim = 0.51371205
jacc_dist = 0.48628795


0.48628795

Now we work with Min-hashing

In [40]:
import java.io.File

val dir = new File("dataset_abstracts")
val files = dir.listFiles() //379
val maxSize = 10
val docSize = if(files.size < maxSize) files.size else maxSize

val docs = files.map(f => sc.textFile(f.getPath() ).map(line=>line.replaceAll("(\\s)+", " ")).collect().reduce(_ + _))
val docsRDD = sc.parallelize(docs).cache()
val abstracts = docsRDD.map(doc => doc.split("Abstract :")).filter(a => a.size > 1).map(a => a(1))

val k = 5
val k_gram1 = abstracts.map(a => a.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), "")))
val kgramRDD = sc.parallelize(k_gram1)
val filtered = kgramRDD.distinct()

Name: Unknown Error
Message: <console>:55: error: type mismatch;
 found   : org.apache.spark.rdd.RDD[List[(Int, String)]]
 required: Seq[?]
Error occurred in an application involving default arguments.
       val kgramRDD = sc.parallelize(k_gram1)
                                     ^

StackTrace: 

In [33]:

//abs.collect().foreach(println)
//println(abs.count())
//println(abs.take(1)(0))

377
 This research is part of an on-going program by the  principal investigator and associates. Topics in the following  areas are to be considered: (1) controlled Markov diffusions  and nonlinear PDEs; (2) asymptotic properties of nearly  deterministic Markov processes; (3) financial economics  applications; (4) singular stochastic control; (5) computational  methods in stochastic control; (6) stochastic calculus of  variations; (7) nonlinear estimation. Analytical methods based  on viscosity solution techniques for nonlinear differential  equations as well as probabilistic methods will be studied.  These theoretical studies are the basis for applied problems  ranging from decisions at the stock market level to the control  of spaceships.


docsRDD = ParallelCollectionRDD[4606] at parallelize at <console>:37
abstracts = MapPartitionsRDD[4607] at map at <console>:39
abs = MapPartitionsRDD[4609] at map at <console>:40


MapPartitionsRDD[4609] at map at <console>:40

In [52]:
//Min-hashing

val docSize = 10
val docs = 



val p = 1073676287
val m = p + 1
def hashThis(a:Long, b:Long, x:Long): Long = {
    ((a*x + b) % p ) % m
}

//TODO: Outer loop: foreach document in set of documents
var doc_signature:Array[Long] = new Array[Long](100)
for(i<-0 to 99){
    
    val r = new scala.util.Random(i)
    val a2 = r.nextInt(p-1)
    val a = if(a2%2==0) a2+1 else a2
    val b = r.nextInt(p-1)
    
    for(d in doc){
        val min = d.minhashThis()
    }
    
    val min = filtered1.map(s => s._1.toLong).reduce((x,y) => hashThis(a, b, x) min hashThis(a, b, y) )
    
    doc_signature(i) = min
    
}

p = 1073676287
m = 1073676288
doc_signature = Array(-342689503, -841956040, -99892218, -932095214, -577548700, -585859158, -619577524, -929598556, -903111229, -549820996, -849171872, -1020008119, -775718799, -998815566, -860483878, -600152099, -608684411, -505264634, -1026611494, -460002885, -901909936, -819056087, -819216546, -864331178, -725024484, -454154236, -296771822, -957145494, -798275923, -574231678, -1037749267, -1009271244, -975740647, -682134308, -770975105, -679068068, -1021970304, -239090677, -775895460, -431311941, -497539779, -412289549, -804483231, -385354347, -788001867, -1036339276, -541172357, -1054658366, -917418843, -686658522, -431932535, -799241785, -784821473, -371596209, -1013243213, -435818998, -...


hashThis: (a: Long, b: Long, x: Long)Long


Array(-342689503, -841956040, -99892218, -932095214, -577548700, -585859158, -619577524, -929598556, -903111229, -549820996, -849171872, -1020008119, -775718799, -998815566, -860483878, -600152099, -608684411, -505264634, -1026611494, -460002885, -901909936, -819056087, -819216546, -864331178, -725024484, -454154236, -296771822, -957145494, -798275923, -574231678, -1037749267, -1009271244, -975740647, -682134308, -770975105, -679068068, -1021970304, -239090677, -775895460, -431311941, -497539779, -412289549, -804483231, -385354347, -788001867, -1036339276, -541172357, -1054658366, -917418843, -686658522, -431932535, -799241785, -784821473, -371596209, -1013243213, -435818998, -...